### מטלה בכריית נתונים
#### :מגישים
##### תהל חרמון - 325053221
##### איתמר חושן - 208728352

In [1]:
import requests

In [2]:
response = requests.get("https://www.ad.co.il/car?sp261=13910&rp264=2017,2022")

In [3]:
response.status_code

200

In [4]:
print(response.content.decode('utf-8'))

<!DOCTYPE html>
<html lang="he" dir="rtl" data-bs-theme="">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    
    
    <title>רכבים פרטיים של קיה יד שניה למכירה - ad</title>
    <meta name="description" property="og:description" content="כל הדגמים של קיה ועוד מגוון רכבים פרטיים נמצאים פה בקליק, הכנסו להתרשמות." />
    <link rel="next" href="/car?sp261=13910&amp;rp264=2017,2022&amp;pageindex=2" />
    <link rel="canonical" href="https://www.ad.co.il/car?sp261=13910&amp;rp264=2017,2022" />
        <meta property="og:image" content="https://www.ad.co.il/images/logo/ad.png" />


    <link rel="apple-touch-icon" sizes="57x57" href="/images/icons/apple-icon-57x57.png" />
    <link rel="apple-touch-icon" sizes="60x60" href="/images/icons/apple-icon-60x60.png" />
    <link rel="apple-touch-icon" sizes="72x72" href="/images/icons/apple-icon-72x72.png" />
    <link rel="apple-touch-icon" sizes="76x76" href="/images/icons/apple-icon-76

In [5]:
from bs4 import BeautifulSoup

In [6]:
url = 'https://www.ad.co.il/car?sp261=13910&rp264=2017,2022'
html = requests.get(url)
soup = BeautifulSoup(html.content,'html.parser')

In [7]:
page_links = soup.find_all('a', class_='page-link')
div_pages = soup.find_all('div', class_='px-3 py-2')
page_numbers = [link.get_text().strip() for link in page_links if link.get_text().strip().isdigit()]
div_page_numbers = [div.get_text().strip().split()[-1] for div in div_pages if div.get_text().strip().startswith('עמוד')]

total_pages = int(max(page_numbers + div_page_numbers))

print("Total number of pages:", total_pages)

Total number of pages: 3


In [8]:
def Findprice(df, counter, soup, a, url_ad):
    price = soup.find_all('h2', class_='card-title')[1].text.strip() if len(soup.find_all('h2', class_='card-title')) > 1 else a
    try:
        price = price[:-1]  
        price = price.replace(',', '') 
        price = float(price) 
    except ValueError:
        price = a
        print(f"Error converting price: {price}" + url_ad)
            
    df.loc[counter, 'Price'] = price

def hand(tds, i, df, counter):
    try:
        hand = int(tds[i + 1].text.strip())
    except (ValueError, IndexError):
        hand = a
    df.loc[counter, 'Hand'] = hand

def gear(tds, i, df, counter):
    try:
        gear = tds[i + 1].text.strip()
    except IndexError:
        gear = a
    df.loc[counter, 'Gear'] = gear

def cap_engine(tds, i, df, counter):
    try:
        Capacity_Engine = int(tds[i + 1].text.strip().replace(',', ''))
    except (ValueError, IndexError):
        Capacity_Engine = a
    df.loc[counter, 'Capacity_Engine'] = Capacity_Engine

def eng_type(tds, i, df, counter):
    try:
        Engine_Type = tds[i + 1].text.strip()
    except IndexError:
        Engine_Type = a
    df.loc[counter, 'Engine_Type'] = Engine_Type

def km(tds, i, df, counter):
    try:
        Km = int(tds[i + 1].text.strip().replace(',', ''))
    except (ValueError, IndexError):
        Km = a
    df.loc[counter, 'Km'] = Km
    
def test(tds, i, df, counter):   
    try:
        Test = tds[i + 1].text.strip()
        test_date = datetime.strptime(Test, "%m/%Y").date()
        current_date = datetime.now().date()
        days_left = (test_date - current_date).days
    except (ValueError, IndexError):
        days_left = a
    df.loc[counter, 'Test'] = days_left

def color(tds, i, df, counter):
    try:
        color = tds[i + 1].text.strip()
    except IndexError:
        color = a
    df.loc[counter, 'Color'] = color

def pre_owe(tds, i, df, counter):
    try:
        Prev_ownership = tds[i + 1].text.strip()
    except IndexError:
        Prev_ownership = a
    df.loc[counter, 'Prev_ownership'] = Prev_ownership
    
def cur_owe(tds, i, df, counter):
    try:
        Curr_Ownership = tds[i + 1].text.strip()
    except IndexError:
        Curr_Ownership = a
    df.loc[counter, 'Curr_Ownership'] = Curr_Ownership
    
def area(tds, i, df, counter):
    try:
        Area = tds[i + 1].text.strip()
    except IndexError:
        Area = a
    df.loc[counter, 'Area'] = Area

def city(tds, i, df, counter):
    try:
        City = tds[i + 1].text.strip()
    except IndexError:
        City = a
    df.loc[counter, 'City'] = City

In [17]:
import pandas as pd
import time as tm
import numpy as np
from datetime import datetime
def get_details(soup, df, counter, page_num, url_ad):
    a = np.nan
    df.loc[counter, 'Page_num'] = page_num
    df.loc[counter, 'Manufacturer'] = soup.find("h2", class_="card-title").text[:3] if soup.find("h2", class_="card-title") else a
    df.loc[counter, 'Year'] = int(soup.find("p", class_="card-text").text.strip() if soup.find("p", class_="card-text") else a)
    df.loc[counter, 'Model'] = soup.find("h2", class_="card-title").text[4:].strip() if soup.find("h2", class_="card-title") else a
    df.loc[counter, 'Pic_Num'] = len(soup.find_all("img", class_="desktop-thumbnail bg-video"))
    cre_date_t = soup.find("div", class_="px-3").text.strip()[13:] if soup.find("div", class_="px-3") else a
    df.loc[counter, 'Cre_date'] = datetime.strptime(cre_date_t, '%d/%m/%Y').date() if cre_date_t else a
    reps = soup.find_all('div', class_='px-3')
    for rep in reps:
        if 'תאריך הקפצה אחרון:' in rep.text:
            Repub_date = rep.text.split(': ')[1]
    df.loc[counter, 'Repub_date'] =  datetime.strptime(Repub_date, '%d/%m/%Y').date() if Repub_date else a
    df.loc[counter, 'Description'] = soup.find("p", class_="text-word-break").text.strip() if soup.find("p", class_="text-word-break") else a
    
    Findprice(df, counter, soup, a, url_ad)
    
    columns = ['Hand', 'Gear', 'Capacity_Engine', 'Engine_Type', 'Km', 'Test', 'Color', 'Prev_ownership', 'Curr_Ownership', 'Area', 'City']
    tds = soup.find_all('td')
    i = 0

    while i < len(tds):
        td_text = tds[i].text.strip()
                                  
        if "יד" in td_text:
            hand(tds, i, df, counter)
            i += 2
        elif "ת. הילוכים" in td_text:
            gear(tds, i, df, counter)
            i += 2
        elif "נפח" in td_text:
            cap_engine(tds, i, df, counter)
            i += 2
        elif "סוג מנוע" in td_text:
            eng_type(tds, i, df, counter)
            i += 2
        elif "ק\"מ" in td_text:
            km(tds, i, df, counter)
            i += 2
        elif "טסט עד" in td_text:
            test(tds, i, df, counter)
            i += 2
        elif "צבע" in td_text:
            color(tds, i, df, counter)
            i += 2
        elif "בעלות קודמת" in td_text:
            pre_owe(tds, i, df, counter)
            i += 2
        elif "בעלות נוכחית" in td_text:
            cur_owe(tds, i, df, counter)
            i += 2
        elif "אזור" in td_text:
            area(tds, i, df, counter)
            i += 2
        elif "עיר" in td_text:
            city(tds, i, df, counter)
            i += 2
        else:
            i += 1
    return

In [10]:
counter = 0
urls = []
df = pd.DataFrame(columns=[
    'Manufacturer', 'Year', 'Model', 'Hand','Km', 'Gear', 'Capacity_Engine', 
    'Engine_Type','Color', 'Test', 'Prev_ownership', 'Curr_Ownership', 'Area', 'City', 'Price','Cre_date', 'Repub_date', 'Pic_Num'
])

url_base_ad = 'https://www.ad.co.il/ad/'
counter = 0
for page_num in range(1, total_pages + 1):
    url_page = f"{url}&pageindex={page_num}"
    html = requests.get(url_page)
    soup = BeautifulSoup(html.content,'html.parser')
    id_list_not_filter = soup.find_all("div", class_="card-block")
    id_list_filter=[]
    for id in id_list_not_filter:
        id_list_filter.append(id.get("data-id"))
    id_list_filter=id_list_filter[:-3]
    print(id_list_filter)
    print(len(id_list_filter))
    for id in id_list_filter: 
        url_ad = url_base_ad+id
        urls.append(url_ad) #delete bf sumbit
        html = requests.get(url_ad)
        soup = BeautifulSoup(html.content,'html.parser')
        get_details(soup, df, counter, page_num, url_ad)
        counter+=1


['16184257', '16184864', '16179118', '16185639', '16141767', '16061015', '16066711', '16057275', '16056533', '16029508', '16027637', '16014113', '15956191', '15903550', '15883711', '15861158', '15870734', '15825761', '15792821', '15791019', '15900933', '15742288', '15482464', '15645326', '15585698', '15561471', '15481032', '15396321', '15345010', '16120850', '15122978', '14299424', '14852467', '14720642', '14679870', '14679869', '14664057', '14655356', '14651768', '14622260', '14608113', '14559808', '14510692', '14510704', '14500113', '14496834', '14494197', '14489939']
48
['14486303', '14469297', '14451406', '14445203', '14439004', '14430571', '14423176', '14418779', '14408586', '14399038', '14390697', '14389288', '14380455', '14380457', '14371556', '14371559', '14368402', '14368416', '14359691', '14359685', '14325142', '14294051', '14287028', '14280691', '14276946', '14272389', '14272372', '14251968', '14251944', '14227453', '14222840', '14192192', '14192182', '14182711', '14182610',

In [11]:
df

,Manufacturer,Year,Model,Hand,Km,Gear,Capacity_Engine,Engine_Type,Color,Test,Prev_ownership,Curr_Ownership,Area,City,Price,Cre_date,Repub_date,Pic_Num,Page_num,Description
0,קיה,2019,ספיה,2,95000,אוטומטית,1600,דיזל,אפור עכבר,51,ליסינג,פרטית,עכו - נהריה,נהריה,86000.0,2024-05-01,2024-05-17,5,1.0,"קיה ספורטאז טורבו דיזל, חסכונית בדלק, שמורה, מ..."
1,קיה,2020,סטוניק,3,50000,אוטומטית,1600,בנזין,לבן,204,פרטית,פרטית,עמק יזרעאל,מגדל העמק,96000.0,2024-05-06,2024-05-06,1,1.0,רכב חדששששש
2,קיה,2017,פיקנטו,2,160000,אוטומטית,1200,בנזין,כסוף,82,פרטית,פרטית,אשדוד - אשקלון,אשקלון,50000.0,2024-04-15,2024-04-15,5,1.0,"רכב שמור ביותר , יד שניה , 160,000 קילומטראז׳ ..."
3,קיה,2020,פיקנטו,2,34000,אוטומטית,1300,בנזין,לבן שנהב,173,אחר,פרטית,רמלה - לוד,רמלה,64000.0,2024-05-11,2024-05-11,10,1.0,למכירה קיה פקנטו 2020\n\nקיה פקנטו 2020\n6/12/...
4,קיה,2018,פיקנטו,2,91000,אוטומטית,1200,בנזין,כסוף,NaN,פרטית,פרטית,חיפה וחוף הכרמל,עתלית,57000.0,2024-03-19,2024-03-19,5,1.0,קיה פיקנטו כסופה שנת 2018 צורה החדשה רכב שמור ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,קיה,2020,פיקנטו,2,NaN,אוטומטית,1248,בנזין,NaN,NaN,NaN,NaN,באר שבע והסביבה,באר שבע,NaN,2022-07-18,2022-07-18,0,3.0,"רכב שמור שנת 2020 ידיים פרטיות ,ריפודים מעור ט..."
123,קיה,2019,סטוניק,1,20800,אוטומטית,1000,בנזין,NaN,NaN,NaN,NaN,רמת גן - גבעתיים,גבעתיים,92000.0,2022-07-17,2022-07-17,0,3.0,"דגם פרימיום , גימורי עור, מסך מולטימדיה,"
124,קיה,2020,נירו EV,1,17000,אוטומטית,1580,חשמלי,NaN,NaN,NaN,NaN,רמת השרון - הרצליה,רמת השרון,140000.0,2022-07-12,2022-07-12,0,3.0,NaN
125,קיה,2022,פיקנטו,1,6500,אוטומטית,1000,בנזין,NaN,NaN,NaN,NaN,קיסריה והסביבה,אור עקיבא,NaN,2022-06-07,2022-06-07,0,3.0,רכב לבן ויפה ללא שריטה חדש חדש נסע ממש מעט &lt...


In [12]:
df['Year'] = df['Year'].astype('Int64')
df['Hand'] = df['Hand'].astype('Int64')
df['Capacity_Engine'] = df['Capacity_Engine'].astype('Int64')
df['Km'] = df['Km'].astype('Int64')
df['Test'] = df['Test'].astype('Int64')
df['Pic_Num'] = df['Pic_Num'].astype('Int64')
df['Description'] = df['Description'].astype('str')
df['Pic_Num'] = df['Pic_Num'].astype('Int64')
df['Repub_date'] = pd.to_datetime(df['Repub_date'])
df['Cre_date'] = pd.to_datetime(df['Cre_date']).replace('-','/')
df['Prev_ownership'] = df['Prev_ownership'].astype('category')
df['Curr_Ownership'] = df['Curr_Ownership'].astype('category')
df['Gear'] = df['Gear'].astype('category')
df['Engine_Type'] = df['Engine_Type'].astype('category')
df

,Manufacturer,Year,Model,Hand,Km,Gear,Capacity_Engine,Engine_Type,Color,Test,Prev_ownership,Curr_Ownership,Area,City,Price,Cre_date,Repub_date,Pic_Num,Page_num,Description
0,קיה,2019,ספיה,2,95000,אוטומטית,1600,דיזל,אפור עכבר,51,ליסינג,פרטית,עכו - נהריה,נהריה,86000.0,2024-05-01,2024-05-17,5,1.0,"קיה ספורטאז טורבו דיזל, חסכונית בדלק, שמורה, מ..."
1,קיה,2020,סטוניק,3,50000,אוטומטית,1600,בנזין,לבן,204,פרטית,פרטית,עמק יזרעאל,מגדל העמק,96000.0,2024-05-06,2024-05-06,1,1.0,רכב חדששששש
2,קיה,2017,פיקנטו,2,160000,אוטומטית,1200,בנזין,כסוף,82,פרטית,פרטית,אשדוד - אשקלון,אשקלון,50000.0,2024-04-15,2024-04-15,5,1.0,"רכב שמור ביותר , יד שניה , 160,000 קילומטראז׳ ..."
3,קיה,2020,פיקנטו,2,34000,אוטומטית,1300,בנזין,לבן שנהב,173,אחר,פרטית,רמלה - לוד,רמלה,64000.0,2024-05-11,2024-05-11,10,1.0,למכירה קיה פקנטו 2020\n\nקיה פקנטו 2020\n6/12/...
4,קיה,2018,פיקנטו,2,91000,אוטומטית,1200,בנזין,כסוף,<NA>,פרטית,פרטית,חיפה וחוף הכרמל,עתלית,57000.0,2024-03-19,2024-03-19,5,1.0,קיה פיקנטו כסופה שנת 2018 צורה החדשה רכב שמור ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,קיה,2020,פיקנטו,2,<NA>,אוטומטית,1248,בנזין,NaN,<NA>,NaN,NaN,באר שבע והסביבה,באר שבע,NaN,2022-07-18,2022-07-18,0,3.0,"רכב שמור שנת 2020 ידיים פרטיות ,ריפודים מעור ט..."
123,קיה,2019,סטוניק,1,20800,אוטומטית,1000,בנזין,NaN,<NA>,NaN,NaN,רמת גן - גבעתיים,גבעתיים,92000.0,2022-07-17,2022-07-17,0,3.0,"דגם פרימיום , גימורי עור, מסך מולטימדיה,"
124,קיה,2020,נירו EV,1,17000,אוטומטית,1580,חשמלי,NaN,<NA>,NaN,NaN,רמת השרון - הרצליה,רמת השרון,140000.0,2022-07-12,2022-07-12,0,3.0,nan
125,קיה,2022,פיקנטו,1,6500,אוטומטית,1000,בנזין,NaN,<NA>,NaN,NaN,קיסריה והסביבה,אור עקיבא,NaN,2022-06-07,2022-06-07,0,3.0,רכב לבן ויפה ללא שריטה חדש חדש נסע ממש מעט &lt...


In [19]:
# # הגדרת כתובת ה-API
# url = "https://data.gov.il/api/3/action/datastore_search"
# resource_id = "5e87a7a1-2f6f-41c1-8aec-7216d52a6cf6"

# # הגדרת פרמטרים לבקשה
# params = { 
#     "resource_id": resource_id,
#     "limit": 1000,  # הגבלת מספר הרשומות בכל בקשה
#     "offset": 0     # תחילת המידע מאפס
# }

# all_records = []

# while True:
#     # שליחת הבקשה ל-API
#     response = requests.get(url, params=params)
    
#     # בדיקת קבלת תשובה תקינה
#     if response.status_code == 200:
#         data = response.json()
#         records = data['result']['records']
        
#         if not records:
#             break  # אם אין רשומות נוספות, נפסיק את הלולאה

#         all_records.extend(records)  # הוספת הרשומות לרשימה הכללית
#         params["offset"] += 1000     # עדכון ההזחה להמשך המידע

#         print(f"Number of records retrieved so far: {len(all_records)}")
#     else:
#         print(f"Failed to retrieve data: {response.status_code}")
#         break

# print(f"Total number of records received: {len(all_records)}")

# # הדפסת כל הרשומות
# for record in all_records:
#     print(record)


In [20]:

# הגדרת כתובת ה-API
url = "https://data.gov.il/api/3/action/datastore_search"
resource_id = "5e87a7a1-2f6f-41c1-8aec-7216d52a6cf6"

# הגדרת פרמטרים לבקשה
params = {
    "resource_id": resource_id,
    "limit": 1000,  # הגבלת מספר הרשומות בכל בקשה
    "offset": 0     # תחילת המידע מאפס
}

all_records = []

while True:
    # שליחת הבקשה ל-API
    response = requests.get(url, params=params)
    
    # בדיקת קבלת תשובה תקינה
    if response.status_code == 200:
        data = response.json()
        records = data['result']['records']
        
        if not records:
            break  # אם אין רשומות נוספות, נפסיק את הלולאה

        all_records.extend(records)  # הוספת הרשומות לרשימה הכללית
        params["offset"] += 1000     # עדכון ההזחה להמשך המידע

        print(f"Number of records retrieved so far: {len(all_records)}")
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        break

print(f"Total number of records received: {len(all_records)}")

# סינון הנתונים לפי התוצר 'קיה' והשנים 2017 עד 2022
relevant_info = []

for item in all_records:
    if item['tozar'] == 'קיה' and 2017 <= item['shnat_yitzur'] <= 2022:
        item_info = {
            'Manufacturer': item['tozar'],
            'Model': item['kinuy_mishari'],
            'Manufactured_Year': item['shnat_yitzur'],
            'Supply_Score': item['mispar_rechavim_pailim']
        }
        relevant_info.append(item_info)

# יצירת DataFrame מהנתונים הרלוונטיים
api_df = pd.DataFrame(relevant_info)

# הדפסת ה-DataFrame
print(api_df)

# שמירת הטבלה המפולטרת כקובץ CSV
api_df.to_csv('filtered_kia_data.csv', index=False)


Number of records retrieved so far: 1000
Number of records retrieved so far: 2000
Number of records retrieved so far: 3000
Number of records retrieved so far: 4000
Number of records retrieved so far: 5000
Number of records retrieved so far: 6000
Number of records retrieved so far: 7000
Number of records retrieved so far: 8000
Number of records retrieved so far: 9000
Number of records retrieved so far: 10000
Number of records retrieved so far: 11000
Number of records retrieved so far: 12000
Number of records retrieved so far: 13000
Number of records retrieved so far: 14000
Number of records retrieved so far: 15000
Number of records retrieved so far: 16000
Number of records retrieved so far: 17000
Number of records retrieved so far: 18000
Number of records retrieved so far: 19000
Number of records retrieved so far: 20000
Number of records retrieved so far: 21000
Number of records retrieved so far: 22000
Number of records retrieved so far: 23000
Number of records retrieved so far: 24000
N

In [21]:
api_df

,Manufacturer,Model,Manufactured_Year,Supply_Score
0,קיה,STONIC,2020,421
1,קיה,SPORTAGE,2022,84
2,קיה,CEED,2018,1
3,קיה,XCEED,2021,67
4,קיה,CEED,2018,63
...,...,...,...,...
705,קיה,CEED,2018,34
706,קיה,XCEED,2019,0
707,קיה,CEED,2018,0
708,קיה,SELTOS,2020,1
